In [2]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score

In [4]:
# Read in prepped dataset
titanic = pd.read_csv('../data/classification_dataset.csv')

In [7]:
# Define variables
variables = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 
             'Fare', 'Embarked', 'ticket_count',
             'spouse_present', 'cabin_letter', 'cabin_number', 
             'ticket_letter', 'ticket_number', 'sibling_present']

# Define categorical variables for OneHotEncoder
categorical_variables = ['Sex', 'Embarked', 'cabin_letter', 'ticket_letter']

In [8]:
# Set up train test split
X = titanic[variables]
y = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42, stratify=y)

In [ ]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse=False, drop='first'), categorical_variables)
            ],
            remainder='passthrough'
        )),
        ('pf', PolynomialFeatures(interaction_only=True, include_bias=False)),
        ('vt', VarianceThreshold()),
        ('sc')
    ]
)